https://cafe.naver.com/cantsb <br>
씨씨앙 <br>
국내 1위 다이어트 & 영양재 커뮤니티 <br>
407,339명 <br>

https://cafe.naver.com/cosmania <br>
파우더룸 <br>
국내 1위 뷰티 커뮤니티 - 20대 여성, 우리의 주 타겟층 분포 <br>
1,900,607명 <br>

https://cafe.naver.com/imsanbu <br>
맘스홀릭 베이비 <br>
국내 1위 임신,출산,육아 커뮤니티 <br>
2,774,638명

In [1]:
# import 및 접속/로그인
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from urllib.parse import quote
import pandas as pd
import time
import random
import os

driver = webdriver.Chrome('chromedriver.exe') # jpynd 파일과 같은 디렉토리에 있는 chromedriver.exe를 실행.
soup = bs(driver.page_source, 'html.parser')  # beautifulsoup 관련 프레임 변화 코드인 듯. 이해 부족
driver.implicitly_wait(3)                     # ..

driver.get('https://nid.naver.com/nidlogin.login') # 네이버 로그인 url을 접속하는 코드

In [2]:
base_url = 'https://cafe.naver.com/cantsb'
driver.get(base_url)
how_many_page=int(input("몇 페이지를 크롤링하시겠습니까? : "))
#how_many_page = 10
keyward=input("무슨 키워드로 검색하시겠습니까? : ")
#keyward = "치킨"
driver.find_element_by_name('query').send_keys(keyward)
driver.find_element_by_css_selector('div > form > button').click() # 로그인 버튼 클릭

몇 페이지를 크롤링하시겠습니까? : 100
무슨 키워드로 검색하시겠습니까? : 치킨


In [3]:
sub_url = "/ArticleSearchList.nhn?search.clubid=25016228&search.media=0&search.searchdate=all&userDisplay=15&search.option=0&search.sortBy=date&search.searchBy=1&search.query=%C4%A1%C5%B2&search.viewtype=title&search.page="
article_urls = []

for i in range(how_many_page):
    driver.get(base_url+sub_url+'{}'.format(i+1))   
    
    css_selector = "input#topLayerQueryInput"
    search_inpupt_element = driver.find_element_by_css_selector("input#topLayerQueryInput")
    iframe_element = driver.find_element_by_css_selector("iframe#cafe_main")
    driver.switch_to_frame(iframe_element)    
    
    article_list = driver.find_elements_by_css_selector('div.inner_list > a.article')
    article_urls = [ i.get_attribute('href') for i in article_list ]    

    #i+=1                             ## 페이지 반복 조회를 위한 i+=1을 수행합니다.
    print("**crawlin page...%d" %(i+1))  ## 몇 번째 page를 크롤링하는지 현황을 보여줍니다.
    num = 1                          ## 몇 번째 게시물인지 알기 위한 변수 초기화
    
    """
    for article in article_urls:
        print(article)
    """
    
    for article in article_urls:  ## article이라는 변수를 사용하여 
        try:
            driver.get(article)       ## article_urls 배열에 저장된 게시글에 하나씩 방문합니다.

            driver.switch_to_frame('cafe_main')           ## beautifulsoup을 이용하여 article도 frame 변환을 해줍니다.
            soup = bs(driver.page_source, 'html.parser')  ## ...

            title = soup.select('div.tit-box span.b')[0].get_text() ## 게시글의 제목을 title이라는 변수로 저장합니다.
            content_tags = soup.select('#tbody')[0].select('p')     
                      ## #tbody-p로 분류되어 있는 게시글 내용을 content_tags라는 배열로 가져옵니다.
            content = ' '.join([ tags.get_text() for tags in content_tags ])
                      ## 게시글 내용 전체를 띄어쓰기 하나로 구분하여 전체 게시글로 합칩니다.
                      ## 이는 하나의 게시글 안에 여러 문단으로 떨어져 있는 게시글을 하나의 변수로 저장하는 역할을 수행합니다.

            reply_tags = soup.select('#cmt_list')[0].select('span.comm_body')  ## 댓글 목록을 저장하는 역할을 수행합니다.
            reply = ' '.join([tags.get_text() for tags in reply_tags ]) ## 전체 댓글은 reply라는 변수를 이용하여 하나의 변수로 저장합니다.

            print('%d..' %num, end=' ')  ## page 내에서 몇 번째 게시글인지 표시합니다.
            print(title, end=" : ")      ## 제목을 표시합니다.
            print(content)               ## 게시글 내용을 표시합니다.
            print(reply)                 ## 게시글 댓글을 표시합니다.
            num += 1                     ## 페이지 내의 게시글 반복 조회를 위한 num+=1을 수행합니다.
        except:
            print("%d.." %num, end=" ")
            print("가입 회원에게만 공개입니다.")
            num+=1
                
    print('\n')    
    

**crawlin page...1
1.. 가입 회원에게만 공개입니다.
2.. 가입 회원에게만 공개입니다.
3.. 치킨먹고 한바퀴 돌고 집에가요!  : 임산부 동생이랑 약국 갈겸 나와서 한바퀴돌고양상추 파프리카사서 집에 갑니당 ㅋㅋ더 돌고싶지만꼬맹이 둘이 잠도 안자고 엄마 기다리고 있어요 ㅠ 요놈들~~~♡
오.. 저도오늘치킨먹었는데 ㅎ 전 집에서 구워먹었어요 ㅋㅋ 수고하셧어요ㅎㅎ 좋은밤보내세요♡♡ 치킨 맛나게드시고 좋다요♡ 맛있게드시고 소화도 시키시고 굿굿 언넝 자라 요놈들
4.. 치킨 두조각은 살 안찌겠죠! 🍗  : 오늘 저녁은 치킨 두조각에 콜라 반잔 먹었어요양심적으로 하나는 닭가슴살 부분 먹었는데치팅데이라고 안쳐도 되겠죠??다이어트하니까음식 하나하나에 쫄보가 되네용..배는 안차네요😰😰😰🤣🤣🤣🤣
마카롱 네개는 살 찌겠죠! ㅠㅠ 치킨은 그나마 단백질이니까 더 나을거같아요~~ 옴마야 오늘 마카롱도 두개 먹었는데 어찌 아셧는지 깜놀했어요ㅋㅋㅋ 앗ㅋㅋ 제가 먹은거 고해성사 한거였는데ㅋㅋㅋ 메뉴 통했네용 ㅎㅎㅎ 치팅데이까지 안쳐도될듯해요♡ 헤헤 배가 아직도 허전하네요 다이어트 계속해도 위는 줄어들기가 힘든가봐용 위는 늘 살아있어요~~!! ㅎㅎ잠시 줄어있을뿐 두조각정도야! 단백질이죠잉ㅋㅋㅋ 치킨은 바삭한 튀김이 문제죠! 지금 운동하러 가야겟네요 ㅋㅋ 콜라는 빼기 ..두조각은 반찬이라고 넘겨드릴게요ㅋㅋ 그치만 자꾸 이렇게 합리화하면 안되는거 아시죠? 차라리 치팅이라하고 맘편히 든든히 드시기 그대신 지금 나가서 운동 열심히 하고 오려구여 !!ㅋ
5.. 저 마카롱 먹어부렀어요..치킨도..^^ :           퇴근하고 최소 1시간 이상씩 열심히 운동중인데 불금이라 그런지 멀리서 일하는 칭구가 이 근방으로 미팅 왔는데 자기 바로 현지퇴근 할거라며 점심먹자규 해서 ㅋㅋㅋㅋㅋ 저 급 삘 받아서 점심시간에 거래처 핑계 대고 친구랑 대낮부터 치킨에 후식으로 마카롱에 ㅋㅋㅋㅋㅋㅋㅋㅋ 마카롱은 친구가 사와서 안 먹을 수가 없었네요 ㅠㅠ ㅋㅋㅋㅋ 암튼 요즘 다욧 빡세게 하고 있고 얼굴

10.. 어제..치킨먹었어요♡ :   어제 치킨을 냠냠냠냠냠 먹었는데 0.4키로 올라갔어요...흑흑 오늘 열심히 하려구요^^...  공복 싸이클후 아점 먹네요~!! 먹고 좀 뒹굴딍굴 해야겠어요ㅠㅠ                             
담는것도 어찌 이리 예쁘게 담으셨나요 이런걸로 칭찬받을줄이야ㅎㅎ기부니좋으네용♡ 캬 ㅎㅎ넘 맛있게 예쁘게 담으셨다~~~♡ 맛.....있었어요!!ㅋㅋㅋㅋ치킨보단 아니지만....ㅎㅎ 오늘도 홧팅입니다! !^^ 넹!!! 오늘도 화이팅이에용! 치킨 드신거치고 증량 덜하신거아닐까용! 급찐급빠갑씨당ㅋㅋㅋㅋㅋ 맛점되세용 넹넹!! 오늘 빡세게 해야겟어용ㅎㅎ 아우 깔끔하고 이쁘게 담으셨네요^^ 그와중에 눈에들어오는 반숙계란 너무나내스탈ㅠ맛나게드세요♡ 그쵸ㅎㅎ반숙 넘 맛있어요♡♡♡ 줄을 반듯반듯 잘 세우셨당즐점하셨죠? ㅎ 넹넹 배부르네여ㅎㅎ....ㅋㅋㅋ뭐든 줄세우는 버릇아닌 버릇이ㅠ 정갈하네요!! 낼은 내려갈꾸에용 맛점하세용    ㅎㅎ넹 아자아자 화이팅!!!!   
11.. 가입 회원에게만 공개입니다.
12.. 어제는 치킨 : 오늘은피자낼은 짜장면사달라고. 너는 참좋게따다먹으니 ㅋㅋㅋㅋㅋㅋ
ㅡ0ㅡ좋겠다ㅡ엄마가다사줘서ㅡ나는연속으로는안사주는데ㅡㅡㅋ 먹고싶은건 사주는편이러는 좋다ㅡ저는그랬다가는집안거덜날거예요ㅋㅋ둘이입맛이달라서항상먹고싶은게다름ㅡㅡ환장ㅋ 부럽다~~ ㅋㅋㅋ 아가들은 많이 뛰어댕기고 쑥쑥커야하니 이것저것 많이 먹어도 괜찮아용 ㅎㅎㅎ근데 부렵댜 나도 이것저것 다먹고싶다 ㅠㅠ 부럽다 ㅎㅎㅎ 진짜로 그리고 마니도 안먹으니 부롭당❤❤ ㅋㅋ맨날맨날 사줄라면 지갑 탈탈 털리겠는데용 ㅋㅋㅋㅋ 공주는 좋겟다 엄마가 다 사줘서ㅜㅜ 세상부럽~ 난 저때도 많이 못묵음요~ 형제가 4명이라 먹는거땜시 자주 싸움요ㅋㅋㅋ 부러운 공주ㅋㅋ
13.. 가입 회원에게만 공개입니다.
14.. 치킨 : 버려야하나아님. 냉동 가능한가여?입짧은것들하고는.....1인1덝이 치킨에대한예의이거늘.....
저거 살만 찢어서 카레에 넣거나 파스타에 넣음 정말 맛있어요치킨은 기본 간

5.. 다이어트 치킨..... :   https://rankingdak.com/new_event/manwoojul.php    치킨의 유혹에 끌려.. 에라이!!!!                

6.. 300칼로리 치킨케밥라이스 풀무원 잇슬림 도시락 후기 :      <본인 인증샷> (사진첨부)    <제품 요약> ※ 제품에 대한 한줄 요약을 해주세요.(사진첨부 X) 가장 친한 친구에게 제품을 설명하는 방식으로 작성해주세요. 1. 제품명 : 치킨케밥라이스 + 치즈볼앤후르츠 2. 제품 효과 : 다이어트 도시락! 3. 가격 : 1주 300덮밥+후르츠컵 배달이 52500원 4. 맛 : 치킨케밥 5. 먹는 방법 : 전자렌지 1분~1분 30초 6. 혹시 주의 사항이 있나요?  없습니다 7. 재구매할 의향이 있나요?  네! 8. 이런 분들에게 추천한다 !  회사에 도시락을 싸가시는 분 적극 추천   <후기> ※ 사진을 포함한 자세한 후기를 작성해주세요. 오늘은~~~~~~ 받자마자 기분이 너무 좋았습니다.. 이름만 봐도.. 치킨케밥라이스.. 거기에 치즈볼..치즈볼이 들어있는 간식이래요...흐흐 새벽에 받자마자 너무너무 먹고싶어서 계속 점심만 기다렸습니다..ㅠㅠ(아침안먹고 점심이 첫끼라서..) 개봉박두! 치킨케밥 라이스!! 칼로리도 298칼로리! 어제도 풀무원 잇슬림 도시락을 먹었는데 칼로리가 워낙 낮다보니 하루 천칼로리를 못먹더라구요 그래서 오늘은..닭가슴살 소세지 한개 추가!! 하루 1000칼로리는 먹어야겠단 생각때문에 닭가슴살을 한개 추가했는데.. 데워보니 안에 닭가슴살이 있었음...ㅋㅋㅋ 덕분에 먹고나서 엄청 배불렀습니다;; 음 맛은..지난번 먹음 씨푸드빠에야랑 좀 비슷한데 카레맛이 좀 덜한 맛이었습니다. 지금까지 먹은 도시락중에 이번것이 제일 맛있었습니다 이름효과인가...치킨..사실 치킨은 아니지만.. 닭가슴살케밥라이스였으면 실망했을수도... 치킨케밥라이스를 먹고 배가 너무 불렀지만... 오늘만큼은 간식을 바로 먹었습니다 왜냐..이름이 치즈볼앤후르츠 라서요.. 치즈

13.. 가입 회원에게만 공개입니다.
14.. 가입 회원에게만 공개입니다.
15.. 가입 회원에게만 공개입니다.


**crawlin page...6
1.. 가입 회원에게만 공개입니다.
2.. 치킨 처갓집이랑 비비큐중에  : 비비큐 기본치킨 또는 아이랑 먹을메뉴중 하나랑 처갓집 더화이트치킨 두개중에 어떤게 맛있을까요?? 쿠폰적용땜에 두개중에 고르려고 하거든요.  두개 다 드셔보신분 알려주세요 ~~~~
둘다 안먹어봤는데... 비비큐가 끌리네용 ㅋㅋㅋ 오잉 ~~   하나만 시켜야 하는데 ㅎㅎ 둘다안먹어봤는데ㅠ.ㅠ 처갓집! 이런 ㅎㅎㅎ  두개시키면 폭식인데 ㅋ 처갓집! 처갓집은 더화이트밖에 할인이 안되요 ~ 드셔봤나요 ^^ 아뇨ㅡㅜ 그냥 처갓집이 치킨이맛있어서 네 ^^ 처갓집으로 왔어요 ^^
3.. 가입 회원에게만 공개입니다.
4.. 아침 치킨..! : 동생이 먹다 남긴 치킨!♡에프에 돌려서 상추랑 같이 야무지게 먹습니당ㅋㅋㅋ10개월만에 먹는 치킨이에요ㅠㅠ맛있다!!ㅋㅋㅋㅋ
맛나게드세요♡ 넹♡ 언제먹어도 치킨은 맛있네요ㅠㅠ 모닝치킨도 삼겹살도 가능한 우리들 입니다 ㅎ ㅋㅋㅋ안하고 못할뿐이지요ㅋㅋ 그럼요ㅋㅋㅋ아침에 기름진거 충분히 소화가능!!ㅋㅋㅋ 착한동생덕에 아침부터 푸짐하시네요ㅋㅋ맛나게드시고 즐겁게하루시작하세요~ 네 ㅎㅎ 맛있게 먹고 즐겁게 하루 시작해요♡ 동생아 고마오 ㅎㅎㅎ 슈스님 맛나게드세욤♡ 동생이 제 생각해서 많이 남겼더라고요 ㅎㅎ맛있게 먹고 있어요♡ 모닝 치킨 맛나게 드세요~^^ 넵 ㅎㅎ 맛있게 먹었어요^^ 우와..10개월만에 드시다니 대단하세요 그동안 치킨 먹고 싶은거 꾹 참았었어요오늘 드디어 먹었네요ㅋㅋ 맛있게드세요^^ 맛나게 순삭했어요 ㅎㅎ 우왕 10개월.. 너무 오래안드셨당 ㅠ 다이어트 기간 중에 아예 안먹었..ㅋㅋㅋ한번 입터지면 계속 당길까봐 참았는데 이제 유지 시작했으니까 먹고 싶으면 먹을꺼에요 ㅎㅎ 치킨 좋아요★♥ 치킨은 사랑♡ 10개월만에 대단하시당 ㅎㅎ 다들 너무 멋지게 사시세요~     헐 완전 맛난거>.< 맛있게드세요 ㅎㅎㅎ 튀김옷 짱

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=74.0.3729.157)
  (Driver info: chromedriver=2.35.528161 (5b82f2d2aae0ca24b877009200ced9065a772e73),platform=Windows NT 10.0.17763 x86_64)


In [4]:
import nltk
from konlpy.tag import Twitter  ## konlpy.tag 형태소 분석기를 사용합니다.
tw = Twitter()                  ## 이를 tw 변수에 할당합니다.

text= open('test_씨씨앙_original.txt').read()  ## 형태소 분석을 하고 싶은 데이터.txt를 jpynd 파일과 같은 디렉토리에 옮긴 뒤,
                                        ##텍스트를 열어서 text라는 변수에 저장합니다.

# 명사 추출
## sentences : 문장
## nouns : 명사
## pos : 형태소

nouns = tw.nouns(text)          ## text의 내용을 형태소 분석하여 명사만 추출한 뒤, 이를 nouns 변수에 저장한다.

# stop_words = ['가','나','다']                                 ## 이 코드에서 사용하지 않았지만, 원하는 키워드를 제거할 수 있습니다.
# nouns = [word for word in nouns if word not in stop_words]    ## 가, 나, 다 키워드는 제거하겠다를 표시하는 코드입니다.

nltk_text = nltk.Text(nouns)               ## text에서 추출한 nouns 변수의 빈도수를 수치화하는 코드입니다.
data = nltk_text.vocab().most_common(500)  ## 상위 몇 개의 명사를 추출할지 정하는 코드입니다. 저희는 상위 500개 데이터를 추출하였습니다.
data = dict(data)                          ## 각각의 랭킹된 명사를 집합으로 만들었습니다.
print(data)                                ## 이를 출력하여 사용하였습니다.

{'치킨': 259, '저': 81, '맛': 79, '오늘': 61, '가입': 42, '회원': 42, '공개': 42, '전': 42, '요': 41, '샐러드': 33, '진짜': 29, '아침': 29, '더': 28, '조각': 28, '용': 28, '운동': 28, '닭': 27, '너무': 26, '저녁': 25, '다이어트': 25, '화이팅': 24, '살': 22, '교촌': 22, '것': 22, '제품': 22, '안': 21, '거': 21, '맥주': 21, '낼': 20, '치즈볼': 20, '때': 19, '내일': 19, '과일': 19, '분': 19, '개': 18, '칼로리': 18, '가슴': 17, '제': 17, '어제': 17, '또': 16, '봐': 16, '좀': 16, '식단': 16, '소스': 15, '점심': 14, '신랑': 14, '감량': 14, '하나': 13, '정도': 13, '와': 13, '왜': 13, '집': 12, '지금': 12, '이제': 12, '마리': 12, '마늘': 12, '한번': 12, '해': 11, '튀김': 11, '사진': 11, '역시': 11, '콤보': 11, '처갓집': 11, '입': 11, '유혹': 11, '도시락': 11, '욧': 10, '일차': 10, '양념': 10, '파': 10, '허니': 10, '네': 10, '애': 10, '후르': 10, '사과': 10, '치': 9, '날': 9, '공복': 9, '하루': 9, '이번': 9, '일': 9, '사랑': 9, '꼭': 9, '그냥': 9, '교촌치킨': 9, '라이스': 9, '풀무원': 9, '음식': 8, '시간': 8, '냄새': 8, '네네': 8, '통닭': 8, '뿌': 8, '배달': 8, '시': 8, '참고': 8, '생각': 8, '몸무게': 8, '저희': 8, '시저': 8, '야채': 8, '프로': 8, '오': 7, '마카롱': 7, '단백질': 7

In [ ]:
# import 및 접속/로그인
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from urllib.parse import quote
from urllib.request import urlopen
import pandas as pd
import time
import random
import os

In [ ]:
url = 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
html = urlopen(url)
soup = BeautifulSoup(html, "lxml")
print(str(soup)[:1000])

In [ ]:
html_title = soup.title
html_title

In [ ]:
tag_name = soup.title.name
tag_name

In [ ]:
p_tag_name = soup.title.parent.name
p_tag_name

In [ ]:
tag_text = soup.title.string
tag_text

In [ ]:
soup.title.getText()

In [ ]:
print(tag_text)

In [ ]:
soup.div

In [ ]:
div_tags = soup.find_all('div')

In [ ]:
len(div_tags)

In [ ]:
div_tags[0]

In [ ]:
print(soup.find_all('div', 'sammy'))

In [ ]:
tmp = soup.find_all('div', 'sammyRank')
tmp[0].get_text()

In [ ]:
rank = []

for i in listOfsoup:
    rank.append(i.find('div', 'sammyRank').get_text())
    
rank[:10]

In [ ]:
import re

In [ ]:
tmpString = str(listOfsoup[0].find("a").get_text())
tmpString

In [ ]:
re.split(('\n|\r\n'), tmpString)

In [ ]:
re.split(('\n|\r\n'), tmpString)[0]
re.split(('\n|\r\n'), tmpString)[1]

In [ ]:
listOfsoup[0].find("a")

In [ ]:
rank = []
mainMenu = []
cafeName = []
urlAdd = []

for item in listOfsoup:
    rank.append(item.find('div', 'sammyRank').get_text())
    
    tmpString = str(item.find("a").get_text())
    tmp = [tmpLine for tmpLine in re.split(('\n|\r\n'), tmpString)]
    mainMenu.append(tmp[0])
    cafeName.append(tmp[1])
    
    tmp2 = item.find("a")["href"]
    if tmp2[:4] != 'http':
        urlAdd.append('http://www.chicagomag.com'+item.find("a")["href"])
    else :
        urlAdd.append(item.find("a")["href"])

In [ ]:
urlAdd[:10]

In [ ]:
import pandas as pd

data = {'Rank':rank, 'Menu':mainMenu, 'Cafe':cafeName, 'URL':urlAdd}
df = pd.DataFrame(data)

In [ ]:
df.head(10)

In [ ]:
import pandas as pd

data = {'Rank':rank, 'Menu':mainMenu, 'Cafe':cafeName, 'URL':urlAdd}
df = pd.DataFrame(data, columns=['Rank', 'Cafe', 'Menu', 'URL'])
df.head(10)

In [ ]:
df.to_csv('data/exam-WebParsing.csv', sep=',', encoding = 'UTF-8')